In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv(r'/content/drive/MyDrive/train_20.csv', encoding='utf_8_sig')
test = pd.read_csv(r'/content/drive/MyDrive/test_20.csv', encoding='utf_8_sig')


In [4]:
train

,label,text,diff
0,negative,Fix macro names - related to #448,diff --git a/src/ciphers/aes/aes.c b/src/ciphe...
1,negative,9.0.4,diff --git a/package.json b/package.json\ninde...
2,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,negative,test: incr. disk size for TEST 35 ISCSI-MULTI,diff --git a/test/TEST-35-ISCSI-MULTI/test.sh ...
4,negative,git-svn-id: https://plugins.svn.wordpress.org/...,diff --git a/inc/define.php b/inc/define.php\n...
5,negative,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
6,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
7,positive,Normalize resource URL in ResourceServlet\n\nI...,diff --git a/spring-webmvc/src/main/java/org/s...
8,negative,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
9,negative,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...


In [5]:
len(train)

50

In [6]:
train['label'].value_counts()

negative    31
positive    19
Name: label, dtype: int64

In [7]:
test['label'].value_counts()

negative    6316
positive    3746
Name: label, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,negative,Merge pull request #371 from rhenium/ky/hmac-e...,diff --git a/ext/openssl/extconf.rb b/ext/open...,"[-0.007123922, -0.023825081, 0.080380954, -0.0..."
1,positive,Additional check when unpacking archives. Cont...,diff --git a/hadoop-common-project/hadoop-comm...,"[0.040803947, 0.3810564, -0.04827068, 0.059583..."
2,negative,Allow lists of trackingid footers\n\nGerrit al...,diff --git a/templates/gerrit.config.erb b/tem...,"[-0.08971207, 0.21257299, -0.020339478, 0.1116..."
3,negative,Add test for opening a NeXT sound file with ze...,diff --git a/ChangeLog b/ChangeLog\nindex aaa5...,"[-0.1575542, 0.05420924, 0.026407795, -0.03680..."
4,positive,git-prompt.sh: don't put unsanitized branch na...,diff --git a/contrib/completion/git-prompt.sh ...,"[-0.10036319, -0.06585033, -0.06453756, 0.0564..."
...,...,...,...,...
10057,positive,Fix hostname verification using the deprecated...,diff --git a/rt/transports/http/src/main/java/...,"[-0.09865726, 0.1240466, 0.018913323, -0.12461..."
10058,negative,mod_mam: Add missing semicolon,diff --git a/src/mod_mam.erl b/src/mod_mam.erl...,"[-0.009726497, -0.116688214, -0.024219586, -0...."
10059,negative,Create an initial project,diff --git a/.gitignore b/.gitignore\nnew file...,"[-0.21997096, 0.08384448, -0.060743425, -0.108..."
10060,negative,Merge pull request #501 from timgates42/bugfix...,diff --git a/src/config.c b/src/config.c\ninde...,"[0.01273859, -0.052862614, -0.011895754, 0.016..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.7617769827072153
Logistic Regression Precision: 0.7577239120843295
Logistic Regression Recall: 0.7617769827072153
Logistic Regression Recall: 0.7559174479450217
